<a href="https://colab.research.google.com/github/GHooN99/My_study2/blob/master/colab/kaggle_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle competitions download -c regression-cabbage-price

  0% 0.00/187 [00:00<?, ?B/s]
100% 187/187 [00:00<00:00, 319kB/s]
  0% 0.00/61.0 [00:00<?, ?B/s]
100% 61.0/61.0 [00:00<00:00, 57.0kB/s]
  0% 0.00/90.9k [00:00<?, ?B/s]
100% 90.9k/90.9k [00:00<00:00, 30.5MB/s]


In [7]:
!ls

'~.'	       sample_data	   test_cabbage_price.csv
 kaggle.json   submit_sample.csv   train_cabbage_price.csv


In [57]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [58]:
data = pd.read_csv("train_cabbage_price.csv")
data.head(10)

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
5,20100106,-7.3,-11.4,-2.5,0.3,2060
6,20100107,-6.7,-11.2,-1.2,0.0,2140
7,20100108,-5.6,-11.4,1.4,0.0,2140
8,20100109,-3.1,-8.8,1.8,0.1,2140
9,20100110,-1.3,-5.0,2.8,0.0,2140


In [70]:
x_data = data.iloc[:,1:-1]
y_data = data.iloc[:,-1]

,avgTemp,minTemp,maxTemp,rainFall
0,-4.9,-11.0,0.9,0.0
1,-3.1,-5.5,5.5,0.8
2,-2.9,-6.9,1.4,0.0
3,-1.8,-5.1,2.2,5.9
4,-5.2,-8.7,-1.8,0.7
...,...,...,...,...
2917,-3.9,-8.0,0.7,0.0
2918,-1.5,-6.9,3.7,0.0
2919,2.9,-2.1,8.0,0.0
2920,2.9,-1.6,7.1,0.6


In [ ]:
with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(Dense(30,input_dim=4,activation='relu'))
  model.add(Dense(6,activation='relu'))
  model.add(Dense(1))
  optim = optimizers.Adam(learning_rate=0.0025)

  model.compile(optimizer=optim,loss='mse')
  model.fit(x_data,y_data,batch_size=16,epochs=120)

In [72]:
test_data= pd.read_csv("test_cabbage_price.csv")
test_data = test_data.iloc[:,1:]
test_data

,avgTemp,minTemp,maxTemp,rainFall
0,13.5,8.0,19.2,2.6
1,1.1,-3.0,9.5,0.8
2,12.0,5.1,19.6,0.0
3,16.6,11.9,22.5,0.1
4,22.4,17.8,27.4,0.5
5,18.8,13.9,24.4,0.0


In [73]:
test_y_data = model.predict(test_data)

In [76]:
result = pd.DataFrame({'Id':range(0,6),'Expected':test_y_data.flatten()})
result

,Id,Expected
0,0,3291.855713
1,1,3195.288330
2,2,3360.026123
3,3,3434.052002
4,4,3615.235840
5,5,3501.860840


In [78]:
result.to_csv("result.csv",index=False,header=True)

In [80]:
!kaggle competitions submit -c regression-cabbage-price -f result.csv -m "First submit"

100% 83.0/83.0 [00:02<00:00, 28.4B/s]
Successfully submitted to Regression : Cabbage price